In [ ]:
!pip install transformers
!pip install konlpy
!pip install unidic-lite
!pip install mecab-python3
!sudo apt-get install -y curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
transformers.__version__

'4.29.1'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
#transformers의 버전이 4.29.0으로 TFGPT2Model를 사용 TFGPTNeoModel는 4.10.0부터 사용이 가능하다.
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

import re
import string
import nltk
import konlpy
from konlpy.tag import Mecab
mecab = Mecab()

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#네이버 영화 리뷰 데이터셋
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="test.txt")

('test.txt', <http.client.HTTPMessage at 0x7f222dc112a0>)

In [ ]:
train_data = pd.read_table('train.txt')
test_data = pd.read_table('test.txt')

In [ ]:
print('훈련용 리뷰 개수 :',len(train_data))

훈련용 리뷰 개수 : 150000


In [ ]:
print('테스트용 리뷰 개수 :',len(test_data))

테스트용 리뷰 개수 : 50000


In [ ]:
train_data.head() # 상위 5개 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
test_data.head() # 상위 5개 출력

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
def normalize_text(text):
    if not isinstance(text, str):
        return text
    #HTML 태그 제거
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    #이메일 제거
    pattern = r'\S+@\S+'
    text = re.sub(pattern, '', text)

    #전화번호 제거
    pattern = r'\d{2,3}-\d{3,4}-\d{4}'
    text = re.sub(pattern, '', text)

    #링크 제거
    pattern = r'http\S+|www\S+'
    text = re.sub(pattern, '', text)

    # 영어 텍스트 처리
    # 소문자로 변환
    text = text.lower()

    #특수문자 제거
    text = re.sub('[^a-zA-Z0-9가-힣\s]', '', text)

    # 불용어 제거
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if not word in stopwords]
    text = ' '.join(tokens)
    
    # 형태소 분석기를 사용한 토큰화
    tokens = mecab.morphs(text)
    
    # 불용어 제거
    stopwords = ['에', '는', '은', '을', '를', '이', '가', '으로', '로', '에서', '과', '와', '의', '도', '다', '만', '들', '은데', '고', '하다', '다']
    tokens = [word for word in tokens if not word in stopwords]
    # 중복된 단어를 하나로 줄이기
    new_tokens = []
    for word in tokens:
        if len(new_tokens) == 0 or word != new_tokens[-1]:
            new_tokens.append(word)
    text = ' '.join(new_tokens)
    
    return text

In [ ]:
train_data['document'] = train_data['document'].apply(normalize_text)
test_data['document'] = test_data['document'].apply(normalize_text)

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data[train_data['document'] != '']
train_data = train_data.reset_index(drop=True)
print(train_data.isna().sum()) # Null 값이 존재하는지 확인

id          0
document    0
label       0
dtype: int64


In [ ]:
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
test_data = test_data[test_data['document'] != '']
test_data = test_data.reset_index(drop=True)
print(test_data.isna().sum()) # Null 값이 존재하는지 확인

id          0
document    0
label       0
dtype: int64


In [ ]:
print(len(train_data))

149335


In [ ]:
print(len(test_data))

49763


In [ ]:
lengths = [len(x) for x in train_data['document']]
sorted_data = train_data.iloc[np.argsort(lengths)][::-1]
sorted_data

,id,document,label
110245,9352925,어릴 적 영화 티비 방송 되 엄마 보 라고 하 시 며 녹화 해 주 시 이후 녹화 한...,1
100381,9097019,140 맞추 기 드럽 게 힘드 네 진짜 이런 쓰레기 세상 에다 있 네 분리수거 해야...,0
57480,8395425,내 노래 잘 알 거나 그런 건 아닌데 솔직히 합 채 랑 썸 띵 중 썸 띵 떨어진 건...,0
83253,8517243,아니 왜 썸 띵 미워하 냐고요 동영상 보 니깐 김아현 눈 밑 다크 써 클 내려와 잇...,0
126001,7928789,눈뜨 본 생방 중 가장 최악 었 던 프로그램 시청 하 너무 불쾌 해 평점 남 깁니다...,0
...,...,...,...
49962,6645285,쩜,1
14217,7247064,애,0
136961,4814547,좋,1
49964,5158262,임,1


In [ ]:
#사전학습
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',bos_token='<s>', eos_token='</s>', pad_token='<pad>') #kogpt2

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']


In [ ]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]


In [ ]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [ ]:
train_data['document'].apply(len).max()

190

In [ ]:
max_seq_len = 200

In [ ]:
encoded_result = tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역", max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 200


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 149335/149335 [00:31<00:00, 4729.09it/s]


In [ ]:
train_X

array([[    0,  9050,  9267, ...,     3,     3,     3],
       [    0, 19243, 40360, ...,     3,     3,     3],
       [    0, 12371,  9150, ...,     3,     3,     3],
       ...,
       [    0,  9018,  6866, ...,     3,     3,     3],
       [    0, 43357, 10584, ...,     3,     3,     3],
       [    0,  9426, 10584, ...,     3,     3,     3]])

In [ ]:
train_y

array([0, 1, 0, ..., 0, 1, 0], dtype=int32)

In [ ]:
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49763/49763 [00:08<00:00, 5879.39it/s]


In [ ]:
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    0  9050  9267 12711 23971 12870  8262  9063  9873  8084 48213     1
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     

In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=3)

checkpoint_path = 'weights.h5'

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
model.fit(train_X, train_y, epochs=10, batch_size=64, validation_split=0.2,callbacks=[earlystop_callback, cp_callback])

Epoch 1/10
1867/1867 [==============================] - ETA: 0s - loss: 0.4093 - accuracy: 0.8129
Epoch 1: val_accuracy improved from -inf to 0.86406, saving model to weights.h5
1867/1867 [==============================] - 382s 163ms/step - loss: 0.4093 - accuracy: 0.8129 - val_loss: 0.3175 - val_accuracy: 0.8641
Epoch 2/10
1867/1867 [==============================] - ETA: 0s - loss: 0.3021 - accuracy: 0.8722
Epoch 2: val_accuracy improved from 0.86406 to 0.86714, saving model to weights.h5
1867/1867 [==============================] - 255s 136ms/step - loss: 0.3021 - accuracy: 0.8722 - val_loss: 0.3074 - val_accuracy: 0.8671
Epoch 3/10
1867/1867 [==============================] - ETA: 0s - loss: 0.2647 - accuracy: 0.8909
Epoch 3: val_accuracy improved from 0.86714 to 0.86966, saving model to weights.h5
1867/1867 [==============================] - 255s 136ms/step - loss: 0.2647 - accuracy: 0.8909 - val_loss: 0.3145 - val_accuracy: 0.8697
Epoch 4/10
1867/1867 [===========================

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

49/49 [==============================] - 28s 446ms/step - loss: 0.4344 - accuracy: 0.8607
test loss, test acc:  [0.43441641330718994, 0.8606595396995544]


In [ ]:
model.load_weights(checkpoint_path)

# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드
test_loss, test_acc = model.evaluate(test_X, test_y, verbose=1)
test_loss,test_acc

1556/1556 [==============================] - 51s 30ms/step - loss: 0.3441 - accuracy: 0.8657


(0.3441116213798523, 0.8657436370849609)

In [ ]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('제 점수는 10점 만점에 1점입니다')

1/1 [==============================] - 1s 634ms/step
83.03% 확률로 긍정 리뷰입니다.

